In [ ]:
# @title 1. Simple Baseline (TF-IDF)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, hamming_loss
import pandas as pd
from google.colab import drive

# 1. ربط الدرايف
drive.mount('/content/drive')

# 2. إعداد المسار والأعمدة الجديدة (الأبعاد الثلاثة للاستعارة)
# ملاحظة: تأكدنا من المسار الجديد ومن أسماء الأعمدة كما في عينتك
file_path = '/content/drive/MyDrive/pcode/QuranicMetaphor/corpus/processed_data.csv'
target_cols = ['Tasrihiyya', 'Makniyya', 'Asliyya', 'Tabiyya', 'Murashaha', 'Mujarrada', 'Mutlaqa']

# 3. تحميل البيانات
# مفترض أن الملف مفصول بفاصلة عادية (CSV)، إذا كان غير ذلك يمكن تعديل sep
df = pd.read_csv(file_path, sep='\t',encoding='utf-16')


print(f"تم تحميل {len(df)} آية.")
print("الأعمدة المستهدفة:", target_cols)

# 4. تحويل النص إلى أرقام (TF-IDF)
# نستخدم العمود 'Text_Ayah' بدلاً من 'ayah_text'
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['Text_Ayah'].astype(str))
y = df[target_cols].values

# 5. تقسيم البيانات (85% تدريب - 15% اختبار) ليتوافق مع الورقة
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# 6. النموذج الإحصائي
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train, y_train)

# 7. التقييم
predictions = clf.predict(X_test)

print("\n--- نتائج الـ Baseline (TF-IDF) ---")
print(f"Micro F1: {f1_score(y_test, predictions, average='micro'):.4f}")
print(f"Macro F1: {f1_score(y_test, predictions, average='macro'):.4f}")
print(f"Hamming Loss: {hamming_loss(y_test, predictions):.4f}")





Mounted at /content/drive
تم تحميل 1181 آية.
الأعمدة المستهدفة: ['Tasrihiyya', 'Makniyya', 'Asliyya', 'Tabiyya', 'Murashaha', 'Mujarrada', 'Mutlaqa']

--- نتائج الـ Baseline (TF-IDF) ---
Micro F1: 0.8352
Macro F1: 0.4164
Hamming Loss: 0.1413


In [ ]:
# @title 2. AraBERT Model (Isti'aara Classification)

!pip install -q transformers[torch] datasets arabert farasa pyarabic scikit-learn pandas

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from arabert.preprocess import ArabertPreprocessor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss
from datasets import Dataset

# 1. إعدادات المسار
file_path = '/content/drive/MyDrive/pcode/QuranicMetaphor/corpus/processed_data.csv'
model_name = "aubmindlab/bert-base-arabertv2"
# الأبعاد السبعة للاستعارة
target_cols = ['Tasrihiyya', 'Makniyya', 'Asliyya', 'Tabiyya', 'Murashaha', 'Mujarrada', 'Mutlaqa']

# 2. تحميل البيانات
df = pd.read_csv(file_path, sep='\t',encoding='utf-16')


# 3. المعالجة الأولية للنص القرآني باستخدام AraBERT Preprocessor
prep = ArabertPreprocessor(model_name=model_name)
df['processed_text'] = df['Text_Ayah'].apply(lambda x: prep.preprocess(str(x)))

# تحويل الوسوم إلى Float (ضروري لـ BCEWithLogitsLoss)
df['labels'] = df[target_cols].values.astype(float).tolist()

# 4. تقسيم البيانات (استراتيجية 85/15)
train_df, test_df = train_test_split(df[['processed_text', 'labels']], test_size=0.15, random_state=42)

# 5. التوكنايزر
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["processed_text"], truncation=True, padding="max_length", max_length=128)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True)).map(tokenize_function, batched=True)
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True)).map(tokenize_function, batched=True)

# 6. تحميل النموذج (Multi-label)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(target_cols),
    problem_type="multi_label_classification"
)

# 7. دالة المقاييس (نفس المنطق السابق)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(float)
    return {
        "micro_f1": f1_score(labels, predictions, average="micro"),
        "macro_f1": f1_score(labels, predictions, average="macro"),
        "hamming_loss": hamming_loss(labels, predictions)
    }

# 8. إعدادات التدريب
training_args = TrainingArguments(
    output_dir="./arabert_istiara_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8, # كما ذكرنا في الورقة
    num_train_epochs=10, # 10 دورات كافية غالباً
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

print("--- Start Training AraBERT on Metaphor Dimensions ---")
trainer.train()

print("\n--- Final Results for AraBERT ---")
print(trainer.evaluate())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 8.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [07:03<00:00, 569kiB/s]


[2026-02-03 13:25:17,771 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La


--- Final Results for AraBERT ---


{'eval_loss': 0.34524157643318176, 'eval_micro_f1': 0.8652214891611687, 'eval_macro_f1': 0.5161919584608307, 'eval_hamming_loss': 0.11476725521669343, 'eval_runtime': 1.2599, 'eval_samples_per_second': 141.286, 'eval_steps_per_second': 18.256, 'epoch': 10.0}


In [ ]:
# @title 3. CamelBERT-CA Model (Classical Arabic Optimized)

!pip install -q pyarabic

import pandas as pd
import numpy as np
import pyarabic.araby as araby
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss
from datasets import Dataset

# 1. الإعدادات
file_path = '/content/drive/MyDrive/pcode/QuranicMetaphor/corpus/processed_data.csv'
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-ca" # النموذج المتخصص في الفصحى
target_cols = ['Tasrihiyya', 'Makniyya', 'Asliyya', 'Tabiyya', 'Murashaha', 'Mujarrada', 'Mutlaqa']

# 2. تحميل البيانات
df = pd.read_csv(file_path, sep='\t',encoding='utf-16')


# دالة تنظيف خاصة بـ CamelBERT
def preprocess_camel(text):
    text = str(text)
    text = araby.strip_tashkeel(text)
    text = araby.normalize_alef(text)
    return text

df['clean_text'] = df['Text_Ayah'].apply(preprocess_camel)
df['labels'] = df[target_cols].values.astype(float).tolist()

# 3. التقسيم
train_df, test_df = train_test_split(df[['clean_text', 'labels']], test_size=0.15, random_state=42)

# 4. التوكنايزر
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["clean_text"], truncation=True, padding="max_length", max_length=128)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True)).map(tokenize_function, batched=True)
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True)).map(tokenize_function, batched=True)

# 5. النموذج
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(target_cols),
    problem_type="multi_label_classification"
)

# 6. المقاييس
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(float)
    return {
        "micro_f1": f1_score(labels, predictions, average="micro"),
        "macro_f1": f1_score(labels, predictions, average="macro"),
        "hamming_loss": hamming_loss(labels, predictions)
    }

# 7. التدريب
training_args = TrainingArguments(
    output_dir="./camelbert_istiara_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

print("--- Start Training CamelBERT ---")
trainer.train()
print(trainer.evaluate())

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

{'eval_loss': 0.3849734961986542, 'eval_micro_f1': 0.8493919550982226, 'eval_macro_f1': 0.47654346906226613, 'eval_hamming_loss': 0.12921348314606743, 'eval_runtime': 1.2817, 'eval_samples_per_second': 138.881, 'eval_steps_per_second': 17.945, 'epoch': 10.0}


In [ ]:
# @title 4. MARBERT Model (Main Model in Paper)

# هذا هو النموذج الأهم في ورقتك الحالية، وقد تمت إضافته ليتسق مع المنهجية
import pandas as pd
import numpy as np
import pyarabic.araby as araby
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss
from datasets import Dataset

file_path = '/content/drive/MyDrive/pcode/QuranicMetaphor/corpus/processed_data.csv'
model_name = "UBC-NLP/MARBERT" # النموذج الأقوى للهجات والنصوص الصعبة
target_cols = ['Tasrihiyya', 'Makniyya', 'Asliyya', 'Tabiyya', 'Murashaha', 'Mujarrada', 'Mutlaqa']

# تحميل
df = pd.read_csv(file_path, sep='\t',encoding='utf-16')


# معالجة (MARBERT يحب إزالة التشكيل)
def preprocess_marbert(text):
    text = str(text)
    text = araby.strip_tashkeel(text)
    return text

df['clean_text'] = df['Text_Ayah'].apply(preprocess_marbert)
df['labels'] = df[target_cols].values.astype(float).tolist()

# تقسيم
train_df, test_df = train_test_split(df[['clean_text', 'labels']], test_size=0.15, random_state=42)

# توكنايزر
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["clean_text"], truncation=True, padding="max_length", max_length=128)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True)).map(tokenize_function, batched=True)
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True)).map(tokenize_function, batched=True)

# النموذج
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(target_cols),
    problem_type="multi_label_classification"
)

# المقاييس
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(float)
    return {
        "micro_f1": f1_score(labels, predictions, average="micro"),
        "macro_f1": f1_score(labels, predictions, average="macro"),
        "hamming_loss": hamming_loss(labels, predictions)
    }

# التدريب
training_args = TrainingArguments(
    output_dir="./marbert_istiara_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

print("--- Start Training MARBERT (Main Model) ---")
trainer.train()
print(trainer.evaluate())

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

{'eval_loss': 0.4008956253528595, 'eval_micro_f1': 0.8603561387066542, 'eval_macro_f1': 0.5207569833246415, 'eval_hamming_loss': 0.11958266452648475, 'eval_runtime': 1.2029, 'eval_samples_per_second': 147.98, 'eval_steps_per_second': 19.121, 'epoch': 10.0}
